In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [0]:
from google.colab import files
files.upload()

Saving xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl to xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl


In [0]:
!pip uninstall xgboost
!pip install xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl

Uninstalling xgboost-0.90:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/xgboost-0.90.dist-info/*
    /usr/local/lib/python3.6/dist-packages/xgboost/*
    /usr/local/xgboost/libxgboost.so
Proceed (y/n)? y
  Successfully uninstalled xgboost-0.90
Processing ./xgboost-0.81-py2.py3-none-manylinux1_x86_64.whl


In [0]:
dataset = pd.read_csv('/content/drive/My Drive/housing.csv')
dataset = dataset.dropna()
dataset.head(10)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
5,-122.25,37.85,52.0,919.0,213.0,413.0,193.0,4.0368,269700.0,NEAR BAY
6,-122.25,37.84,52.0,2535.0,489.0,1094.0,514.0,3.6591,299200.0,NEAR BAY
7,-122.25,37.84,52.0,3104.0,687.0,1157.0,647.0,3.1200,241400.0,NEAR BAY
8,-122.26,37.84,42.0,2555.0,665.0,1206.0,595.0,2.0804,226700.0,NEAR BAY
9,-122.25,37.84,52.0,3549.0,707.0,1551.0,714.0,3.6912,261100.0,NEAR BAY


In [0]:
datasetNorm = dataset.drop('ocean_proximity', 1)
datasetNorm = ((datasetNorm-datasetNorm.min())/(datasetNorm.max()-datasetNorm.min()))*50
dataset = datasetNorm
dataset.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,10.557769,28.374070,39.215686,1.116537,0.993172,0.447042,1.027791,26.983421,45.113319
1,10.607570,28.267800,19.607843,9.025128,8.573867,3.360520,9.348791,26.901353,35.412328
2,10.507968,28.214665,50.000000,1.863014,1.466480,0.690883,1.447130,23.301403,34.752537
3,10.458167,28.214665,50.000000,1.617580,1.815642,0.777768,1.792468,17.734928,33.639140
4,10.458167,28.214665,50.000000,2.066484,2.164804,0.787578,2.121362,11.538806,33.731923


In [0]:
Y = dataset['median_house_value']
X = dataset.loc[:,'longitude':'median_income']

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
x_train_np = x_train.to_numpy()
x_test_np = x_test.to_numpy()
y_train_np = y_train.to_numpy()
y_test_np = y_test.to_numpy()

In [0]:
import torch
from torch.nn import Conv1d
from torch.nn import MaxPool1d
from torch.nn import Flatten
from torch.nn import Linear
from torch.nn import BatchNorm1d
from torch.nn import Dropout
from torch.nn.functional import relu
from torch.utils.data import DataLoader, TensorDataset

In [0]:

class CnnRegressor(torch.nn.Module):
  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs

    self.input_layer = Conv1d(inputs, batch_size, 1)
    self.max_pool = MaxPool1d(1)
    #self.batch_norm = BatchNorm1d(batch_size)
    self.conv_layer = Conv1d(batch_size, 128, 1)
    #self.batch_norm2 = BatchNorm1d(128)
    self.max_pool2 = MaxPool1d(1)
    self.conv_layer2 = Conv1d(128, 128, 1)
    self.max_pool3 = MaxPool1d(1)
    self.conv_layer3 = Conv1d(128, 128, 1)

    self.flatten_layer = Flatten()
    self.linear_layer = Linear(128, 64)
    #self.linear_layer2 = Linear(1024, 8196)
    #self.linear_layer3 = Linear(8196, 1024)
    #self.linear_layer4 = Linear(1024, 256)
    self.output_layer = Linear(64, outputs)

  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pool(output)
    #output = self.batch_norm(output)
    output = relu(self.conv_layer(output))
    #output = self.batch_norm2(output)
    output = self.max_pool2(output)
    output = relu(self.conv_layer2(output))
    output = self.max_pool3(output)
    output = relu(self.conv_layer3(output))

    output = self.flatten_layer(output)
    output = self.linear_layer(output)
    #output = self.linear_layer2(output)
    #output = self.linear_layer3(output)
    #output = self.linear_layer4(output)
    output = self.output_layer(output)
    return output

'''
class CnnRegressor(torch.nn.Module):
  def __init__(self, batch_size, inputs, outputs):
    super(CnnRegressor, self).__init__()
    self.batch_size = batch_size
    self.inputs = inputs
    self.outputs = outputs

    self.input_layer = Conv1d(inputs, 256, 1)
    self.max_pooling_layer = MaxPool1d(1)
    self.batch_norm = BatchNorm1d(256)
    self.conv_layer = Conv1d(256, 512, 1)
    self.batch_norm1 = BatchNorm1d(512)

    #self.conv_layer2 = Conv1d(512, 1024, 1)
    #self.batch_norm2 = BatchNorm1d(1024)
    
    self.flatten_layer = Flatten()
    #self.linear_layer4 = Linear(1024, 2048)
    #self.linear_layer5 = Linear(2048, 1024)
    #self.linear_layer6 = Linear(1024, 512)
    self.linear_layer = Linear(512, 8192)
    self.linear_layer2 = Linear(8192,4096)
    self.drop = Dropout(0.1)
    self.linear_layer3 = Linear(4096,1024)
    self.linear_layer4 = Linear(1024,128)
    self.output_layer = Linear(128, outputs)

  def feed(self, input):
    input = input.reshape((self.batch_size, self.inputs, 1))
    output = relu(self.input_layer(input))
    output = self.max_pooling_layer(output)
    output = self.batch_norm(output)
    output = relu(self.conv_layer(output))
    output = self.batch_norm1(output)

    #output = relu(self.conv_layer2(output))
    #output = self.batch_norm2(output)

    output = self.flatten_layer(output)
    #output = self.linear_layer4(output)
    #output = self.linear_layer5(output)
    #output = self.linear_layer6(output)
    output = self.linear_layer(output)
    output = self.linear_layer2(output)
    output = self.drop(output)
    output = self.linear_layer3(output)
    output = self.linear_layer4(output)
    output = self.output_layer(output)
    return output
'''

'\nclass CnnRegressor(torch.nn.Module):\n  def __init__(self, batch_size, inputs, outputs):\n    super(CnnRegressor, self).__init__()\n    self.batch_size = batch_size\n    self.inputs = inputs\n    self.outputs = outputs\n\n    self.input_layer = Conv1d(inputs, 256, 1)\n    self.max_pooling_layer = MaxPool1d(1)\n    self.batch_norm = BatchNorm1d(256)\n    self.conv_layer = Conv1d(256, 512, 1)\n    self.batch_norm1 = BatchNorm1d(512)\n\n    #self.conv_layer2 = Conv1d(512, 1024, 1)\n    #self.batch_norm2 = BatchNorm1d(1024)\n    \n    self.flatten_layer = Flatten()\n    #self.linear_layer4 = Linear(1024, 2048)\n    #self.linear_layer5 = Linear(2048, 1024)\n    #self.linear_layer6 = Linear(1024, 512)\n    self.linear_layer = Linear(512, 8192)\n    self.linear_layer2 = Linear(8192,4096)\n    self.drop = Dropout(0.1)\n    self.linear_layer3 = Linear(4096,1024)\n    self.linear_layer4 = Linear(1024,128)\n    self.output_layer = Linear(128, outputs)\n\n  def feed(self, input):\n    input = i

In [0]:
from torch.optim import SGD
from torch.nn import L1Loss
!pip install pytorch-ignite
from ignite.contrib.metrics.regression.r2_score import R2Score

     |████████████████████████████████| 112kB 9.6MB/s 


In [0]:
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

model = LinearRegression()
model.fit(x_train_np, y_train_np)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
pred = model.predict(x_test_np)
mse = mean_squared_error(y_test_np, pred)
print("The model's mean squared error is: "+ str(mse))
r2 = r2_score(y_test_np, pred)
print("The model's R^2 score is: "+ str(r2))

The model's mean squared error is: 53.8711259463841
The model's R^2 score is: 0.6173700254470973


In [0]:
batch_size = 64
model = CnnRegressor(batch_size, X.shape[1], 1)
model.cuda()

CnnRegressor(
  (input_layer): Conv1d(8, 64, kernel_size=(1,), stride=(1,))
  (max_pool): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
  (max_pool2): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer2): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (max_pool3): MaxPool1d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  (conv_layer3): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
  (flatten_layer): Flatten()
  (linear_layer): Linear(in_features=128, out_features=64, bias=True)
  (output_layer): Linear(in_features=64, out_features=1, bias=True)
)

In [0]:
from torch.nn import MSELoss
def model_loss(model, dataset, train=False, optimizer=None):
  performance = L1Loss()
  score_metric = R2Score()
  mse = MSELoss()

  avg_loss=0
  avg_score=0
  avg_mse = 0
  count=0
  for input, output in iter(dataset):
    predictions = model.feed(input)
    loss = performance(predictions, output)
    mse_loss = mse(predictions, output)
    score_metric.update([predictions, output])
    score = score_metric.compute()
    if(train):
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    avg_loss += loss.item()
    avg_score += score
    avg_mse += mse_loss.item()
    count += 1

  return avg_loss/count, avg_score/count, avg_mse/count

In [0]:
epochs = 50
optimizer = SGD(model.parameters(), lr=0.0001)

inputs = torch.from_numpy(x_train_np).cuda().float()
outputs = torch.from_numpy(y_train_np.reshape(y_train_np.shape[0],1)).cuda().float()

tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)
losses = []
scores = []
mses = []
for epoch in range(epochs):
  avg_loss, avg_r2_score, avg_mse = model_loss(model, loader, train=True, optimizer=optimizer)
  losses.append(avg_loss)
  scores.append(avg_r2_score)
  mses.append(avg_mse)
  print("Epoch", epoch+1, ':\n\tLoss=', avg_loss, ':\n\tScore=', avg_r2_score, '\n\tMSE=', avg_mse)


Epoch 1 :
	Loss= 4.988023388619516 :
	Score= 0.6268962422250526 
	MSE= 53.042644089343504
Epoch 2 :
	Loss= 4.982614237654443 :
	Score= 0.6258736067695406 
	MSE= 52.93802713880352
Epoch 3 :
	Loss= 4.975751141005872 :
	Score= 0.6196051565937467 
	MSE= 52.77068498648849
Epoch 4 :
	Loss= 4.970019639707079 :
	Score= 0.6293173768274171 
	MSE= 52.746879876828665
Epoch 5 :
	Loss= 4.957995664372164 :
	Score= 0.6259509524934546 
	MSE= 52.47538267023423
Epoch 6 :
	Loss= 4.958145470712699 :
	Score= 0.6376733602369773 
	MSE= 52.41627512913124
Epoch 7 :
	Loss= 4.952243685254864 :
	Score= 0.6289320486195116 
	MSE= 52.31372297698376
Epoch 8 :
	Loss= 4.9371064700332346 :
	Score= 0.6211469263658836 
	MSE= 51.99204458348891
Epoch 9 :
	Loss= 4.936681010676366 :
	Score= 0.6342463152012106 
	MSE= 51.90996514862659
Epoch 10 :
	Loss= 4.929940708010805 :
	Score= 0.6446178137108559 
	MSE= 51.945389721440336
Epoch 11 :
	Loss= 4.9257166077108945 :
	Score= 0.6299887596589724 
	MSE= 51.77884896970263
Epoch 12 :
	Lo

In [0]:
from torch.nn import Sequential
def get_feature_layer(input):
  model_feature_layer = Sequential(*list(model.children())[:-3])
  pred = model_feature_layer(input)
  return pred

In [0]:
reee = get_feature_layer(2)

TypeError: ignored

In [0]:
import xgboost
def train_xgb(dataset, num_round=20):
  
  param = {'colsample_bytree':0.4,
              'gamma':0,                 
              'learning_rate':0.00007,
              'max_depth':9,
              'min_child_weight':1.5,
              'n_estimators':1000,                                                                    
              'reg_alph':0.75,
              'reg_lambd':0.45,
              'subsample':0.6,
              'seed':42}  
  print(x_train.shape[0]//64)
  batch = x_test_np[0:64].reshape((64, 8,1))
  feat_out = get_feature_layer(torch.from_numpy(batch).cuda().float())
  feat_out = feat_out.cpu().detach().numpy()
  #feat_out = feat_out.reshape((64, 128))
  feat_out = feat_out.reshape((64, 4096))
  #print(feat_out.shape)
  #print(y_test.idxmax(axis=1))
  dtrain = xgboost.DMatrix(feat_out.tolist(), label=y_test[0:64].tolist()) 
  xgb_feature_layer = xgboost.train(param, dtrain, num_round)
  #model_save_name = 'prev_model.model'
  path = F"/content/drive/My Drive/prev_model.model" 
  xgb_feature_layer.save_model(path)
  i = 64
  j = 128
  for k in range(x_train.shape[0]//64-1):
    batch = x_train_np[i:j].reshape((64, 8,1))
    feat_out = get_feature_layer(torch.from_numpy(batch).cuda().float())
    feat_out = feat_out.cpu().detach().numpy()
    #feat_out = feat_out.reshape((64, 128))
    feat_out = feat_out.reshape((64, 4096))
    dtrain = xgboost.DMatrix(feat_out.tolist(), label=y_train[i:j].tolist())
    xgb_feature_layer = xgboost.train(param, dtrain, num_round, xgb_model="/content/drive/My Drive/prev_model.model")
    xgb_feature_layer.save_model(path)
    i += 64
    j += 64
    print(k)
  '''
  reeeee = x_test_np[0:64].reshape((64, 8,1))
  feat_out = get_feature_layer(torch.from_numpy(reeeee).cuda().float())
  feat_out = feat_out.cpu().detach().numpy()
  feat_out = feat_out.reshape((64, 256))
  #print(feat_out.shape)
  #print(y_test.idxmax(axis=1))
  dtrain = xgboost.DMatrix(feat_out.tolist(), label=y_test[0:64].tolist()) 
  xgb_feature_layer = xgboost.train(param, dtrain, num_round)
  '''
  return xgb_feature_layer

In [0]:
temp = train_xgb(loader)

Streaming output truncated to the last 5000 lines.
15
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6 extra nodes, 0 pruned nodes, max_depth=2
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 8 extra nodes, 0 pruned nodes, max_depth=3
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=4
[22:07:27] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra no

In [0]:
def make_pred(data):
  reeeee = x_test_np[0:64].reshape((64, 8,1))
  feat_out = get_feature_layer(torch.from_numpy(reeeee).cuda().float())
  feat_out = feat_out.cpu().detach().numpy()
  #feat_out = feat_out.reshape((64, 128))
  feat_out = feat_out.reshape((64, 4096))
  return feat_out

In [0]:
i = 0
j = 64
avg_r2 = 0
score_metric = R2Score()
for k in range(x_test_np.shape[0]//64-1):
  pred = temp.predict(xgboost.DMatrix(make_pred(x_test_np[i:j]).tolist()))
  outputs = torch.from_numpy(y_test_np[i:j].reshape(y_train_np[i:j].shape[0], 1)).cuda().float()
  pred = np.asarray([pred], dtype=np.float32).reshape((64, 1))
  pred = torch.from_numpy(pred).cuda().float()
  score_metric.update([pred, outputs])
  #if k == 1:
  #  print(pred)
  #  print('===')
  #  print(outputs)
  score = score_metric.compute()
  avg_r2 += score
  i += 64
  j += 64
  print("Batch", k, 'Score', score)
print("score", avg_r2/(x_test_np.shape[0]//64))

Batch 0 Score -1.3630127884550562
Batch 1 Score -1.6101041164910646
Batch 2 Score -1.5601832424214166
Batch 3 Score -1.5601081264548546
Batch 4 Score -1.5020398764401195
Batch 5 Score -1.492493760976565
Batch 6 Score -1.4112776925267423
Batch 7 Score -1.3943910586199002
Batch 8 Score -1.3402674414092801
Batch 9 Score -1.358322030458471
Batch 10 Score -1.3248587879498874
Batch 11 Score -1.3278629587368518
Batch 12 Score -1.3251586210559458
Batch 13 Score -1.3222375534828967
Batch 14 Score -1.3219378145644374
Batch 15 Score -1.3087120804587022
Batch 16 Score -1.3179013854887964
Batch 17 Score -1.3154297642329595
Batch 18 Score -1.3192659421728998
Batch 19 Score -1.320137369372778
Batch 20 Score -1.3349815897441695
Batch 21 Score -1.3525540621994798
Batch 22 Score -1.3556919944219619
Batch 23 Score -1.354553561939499
Batch 24 Score -1.3627073999604105
Batch 25 Score -1.3602073383127031
Batch 26 Score -1.34987293089649
Batch 27 Score -1.3600902740572693
Batch 28 Score -1.3476200163372551
B

In [0]:
import xgboost as xgb
model=xgb.XGBClassifier(colsample_bytree=0.4,
              gamma=0,                 
              learning_rate=0.00007,
              max_depth=9,
              min_child_weight=1.5,
              n_estimators=1000,                                                                    
              reg_alph=0.75,
              reg_lambd=0.45,
              subsample=0.6,
              seed=42)
model.fit(x_train_np, y_train_np)
model.score(x_test_np,y_test_np)

KeyboardInterrupt: ignored

In [0]:
inputs = torch.from_numpy(x_test_np).cuda().float()
outputs = torch.from_numpy(y_test_np.reshape(y_test_np.shape[0], 1)).cuda().float()

tensor = TensorDataset(inputs, outputs)
loader = DataLoader(tensor, batch_size, shuffle=True, drop_last=True)

avg_loss, avg_r2_score, avg_mse = model_loss(model, loader)
print('Loss=', avg_loss, '\nR2Score=', avg_r2_score , '\nMSE=', avg_mse)


Loss= 19.462591791909837 
R2Score= -2.8550968104279275 
MSE= 517.5387461828807
